## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:![figure](./fig/hw4q1.png)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：
2.算法基于Kruskal算法的变体，用于验证是否存在一个生成树，将所有边按权重从小到大排序，遍历排序后的边，依次将不形成环且权重不超过b的边加入生成树，最终检查生成树的最大边权重是否不超过b，并计算该生成树的总权重。

3.使用二分查找在图的边权重范围[min_w, max_w]内，通过二分查找确定一个权重阈值mid_w，验证仅用权重≤mid_w的边是否能连通整个图，若连通，尝试更小的阈值；若不可连通，尝试更大的阈值，当左边界超过右边界时，返回最小可行阈值。

answer:![figure](./fig/hw4q2.png)

In [3]:
#2.
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))

    def find(self, p):
        if self.parent[p] != p:
            self.parent[p] = self.find(self.parent[p])
        return self.parent[p]

    def union(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)
        if rootP != rootQ:
            self.parent[rootQ] = rootP


def kruskal(edges, n, b):
    uf = UnionFind(n)
    bottleneck_edge = 0
    mst_weight = 0

    edges.sort(key=lambda x: x[2])

    for u, v, weight in edges:
        if uf.find(u) != uf.find(v):
            if weight > b:
                return False
            uf.union(u, v)
            mst_weight += weight
            bottleneck_edge = max(bottleneck_edge, weight)

    return bottleneck_edge <= b, mst_weight



edges = [
    (0, 1, 10),
    (0, 2, 6),
    (0, 3, 5),
    (1, 3, 15),
    (2, 3, 4)
]
n = 4
b = 10

exists, weight = kruskal(edges, n, b)
print(f"存在瓶颈生成树: {exists}, 瓶颈生成树的权重: {weight}")

# algorithm of the liear time complexity :边排序时间复杂度为O(mlogm)，遍历所有的边O(m)，总的：O(mlogm)

#3.
class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            self.parent[root_y] = root_x
            return True
        return False


def is_connected(edges, n, max_weight):
    uf = UnionFind(n)
    for u, v, w in edges:
        if w <= max_weight:
            uf.union(u, v)
    roots = set(uf.find(i) for i in range(n))
    return len(roots) == 1


def find_bottleneck_bst(edges, n):
    if n == 0:
        return 0
    min_w = min(w for _, _, w in edges)
    max_w = max(w for _, _, w in edges)

    left, right = min_w, max_w
    answer = max_w

    while left <= right:
        mid = (left + right) // 2
        if is_connected(edges, n, mid):
            answer = mid
            right = mid - 1
        else:
            left = mid + 1

    return answer


edges = [
    (0, 1, 4), (0, 2, 1), (1, 2, 2),
    (1, 3, 5), (2, 3, 3)
]
n = 4
bottleneck = find_bottleneck_bst(edges, n)
print(f"瓶颈生成树的最大边权为：{bottleneck}")

# algorithm of the liear time complexity :若边权范围为常数，时间复杂度为 O(m+n)；否则为 O((m+n)log C)

存在瓶颈生成树: True, 瓶颈生成树的权重: 19
瓶颈生成树的最大边权为：3


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


idea :首先使用Floyd-Warshall算法计算图中所有顶点对之间的最短路径。这个算法通过动态规划的方式，逐步考虑通过每个顶点作为中间点的路径，更新顶点对之间的最短路径。对于每条可能的新道路(u,v)，我们模拟这条道路的建设对最短路径的影响。具体来说创建一个新的距离矩阵new_dist，初始时与原始距离矩阵dist相同,在new_dist中，首先设置新道路(u,v)的距离为l。然后，再次使用Floyd-Warshall算法的思想，通过考虑通过u和v的所有可能路径，更新new_dist中的路径长度。计算建设这条道路后从起点s到终点t的距离减少量,减少量大于当前记录的最大减少量,就更新。

时间复杂度：Floyd-Warshall算法的时间复杂度为O(n^3),对于每条新道路也是O(n^3)，假设有m条新道路，总的：O(m*n^3)

In [4]:
def floyd_warshall(n, graph):
    dist = [[float('inf')] * n for _ in range(n)]
    for i in range(n):
        dist[i][i] = 0
        for j in range(n):
            if graph[i][j] != float('inf'):
                dist[i][j] = graph[i][j]
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist

def max_reduction(n, graph, new_roads, s, t):
    dist = floyd_warshall(n, graph)
    max_reduction = 0
    best_road = None
    for u, v, l in new_roads:
        new_dist = [row[:] for row in dist]
        new_dist[u][v] = l
        new_dist[v][u] = l
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    if new_dist[i][k] + new_dist[k][j] < new_dist[i][j]:
                        new_dist[i][j] = new_dist[i][k] + new_dist[k][j]
        if new_dist[s][t] < dist[s][t]:
            current_reduction = dist[s][t] - new_dist[s][t]
            if current_reduction > max_reduction:
                max_reduction = current_reduction
                best_road = (u, v, l)
    return max_reduction, best_road

n = 4
graph = [
    [0, 3, float('inf'), 7],
    [3, 0, 5, float('inf')],
    [float('inf'), 5, 0, 2],
    [7, float('inf'), 2, 0]
]
new_roads = [
    (0, 2, 4),
    (1, 3, 1)
]
s = 0
t = 3
max_reduction_value, best_road = max_reduction(n, graph, new_roads, s, t)
print(f"最大减少量: {max_reduction_value}")
print(f"对应的城市对: {best_road}")

最大减少量: 3
对应的城市对: (1, 3, 1)


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：将逃离问题建模为一个网络流问题，其中网格的每个单元格是一个顶点，每个单元格的四个邻居之间的连接是边。源点s连接到所有起点，汇点t连接到所有边界上的单元格。建一个图，包含源点s、汇点t和网格中的所有顶点。对于每个起点，添加一条从源点到该点的边，容量为1。对于每个边界点，添加一条从该点到汇点的边，容量为1，网格内部的每条边的容量也为1。用Dinic算法计算从源点s到汇点t的最大流。Dinic算法通过构建分层图和寻找增广路径来迭代地增加网络中的流量。使用广度优先搜索构建分层图，使用深度优先搜索在分层图中寻找从源点到汇点的增广路径，并沿着这些路径推送可能的最大流量，重复构建分层图和寻找增广路径的过程，直到无法找到更多的增广路径。

时间复杂度：O(n^6)
answer:![figure](./fig/hw4q4.png)

In [1]:
from collections import defaultdict, deque

class Dinic:
    def __init__(self, n):
        self.n = n
        self.g = [[] for _ in range(n)]
        self.level = None
        self.it = None

    def add_edge(self, fr, to, cap):
        self.g[fr].append([to, cap, len(self.g[to])])
        self.g[to].append([fr, 0, len(self.g[fr]) - 1])

    def bfs(self, s):
        self.level = [-1] * self.n
        q = deque([s])
        self.level[s] = 0
        while q:
            v = q.popleft()
            for e in self.g[v]:
                if e[1] > 0 and self.level[e[0]] < 0:
                    self.level[e[0]] = self.level[v] + 1
                    q.append(e[0])

    def dfs(self, v, t, f):
        if v == t:
            return f
        for i in range(self.it[v], len(self.g[v])):
            e = self.g[v][i]
            if e[1] > 0 and self.level[v] < self.level[e[0]]:
                d = self.dfs(e[0], t, min(f, e[1]))
                if d > 0:
                    e[1] -= d
                    self.g[e[0]][e[2]][1] += d
                    self.it[v] = i
                    return d
        self.it[v] = len(self.g[v])
        return 0

    def max_flow(self, s, t):
        flow = 0
        while True:
            self.bfs(s)
            if self.level[t] < 0:
                return flow
            self.it = [0] * self.n
            while True:
                f = self.dfs(s, t, float('inf'))
                if f == 0:
                    break
                flow += f

def escape(n, sources):
    s = n * n
    t = s + 1
    flow_network = Dinic(t + 1)

    for source in sources:
        flow_network.add_edge(s, source, 1)

    for i in range(n):
        for j in range(n):
            cur = i * n + j
            if i < n - 1:
                flow_network.add_edge(cur, cur + n, 1)
            if j < n - 1:
                flow_network.add_edge(cur, cur + 1, 1)

    for i in range(n):
        flow_network.add_edge(i, t, 1)
        flow_network.add_edge(n * n - n + i, t, 1)

    for i in range(n):
        flow_network.add_edge(i * n, t, 1)
        flow_network.add_edge((i + 1) * n - 1, t, 1)

    return flow_network.max_flow(s, t)

n = 5
sources = [1, 3, 6, 7]
max_escape_paths = escape(n, sources)
print(f"最大逃离路径数: {max_escape_paths}")

最大逃离路径数: 4
